# Exploratory data anaylsis

## Examine the data

The dataset we are working with is an excel speadsheet with different sheets for each year. What does the data look like?

In [ ]:
import pandas as pd
data_path = "../data/raw/Historical Station Counts by State 2007-2021.xlsx"
df = pd.read_excel(data_path, sheet_name='2020', skiprows=1)
df

In [ ]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df_cleaned = df.dropna(how="all")
    df_cleaned = df_cleaned.rename(columns={"Totald" : "Total"})
    df_cleaned = df_cleaned[["State", "Total"]]
    df_cleaned = df_cleaned.dropna()
    df_cleaned['State'] = df_cleaned['State'].astype('category')
    return df_cleaned

df_total = pd.DataFrame()
for n in range(2007, 2022, 1):
    df = pd.read_excel(data_path, sheet_name=str(n), skiprows=1)
    df_cleaned = clean_data(df)
    df_cleaned["Year"] = n
    df_total = pd.concat([df_total, df_cleaned])

Get some general info about the data

In [ ]:
df_total.info(verbose=True)

In [ ]:
df_total.describe(include='all')

Check how many missing values we have

In [ ]:
df_total.isna().mean()

What are the states?

In [ ]:
df.State.value_counts()

The data is a bit of a mess because of footnotes in the excel spreadsheet. Also, there are 52 states in the data so we can expect the results to be noise - you can see this from the percentile cuts on the `Total` data above.

For simplicity, let's trim down to rows with "Total" in the `State` column. This has the added bonus that it makes the problem univariate.

Let's also make the `Year` column a timestamp and set it to be the index. Hint: When we want to make a prediction we will want to use datetime stamps rather than integer row indices as that is what we would want to use in a predict API

In [ ]:
y = df_total[df['State'] == 'Total']
y = y.drop('State', axis=1)
y.Year = pd.to_datetime(y.Year, format="%Y")
y = y.reset_index(drop=True)
y = y.set_index('Year')
y.plot()

## Fit an auto-regressive time series model


Split the data into train and test. Make sure not to shuffle as we want the time order to be preserved for future forecasting.

In [ ]:
from sklearn.model_selection import train_test_split

y_train, y_test = train_test_split(y, shuffle=False)
y_train
y_test

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
model_ag = AutoReg(endog = y_train, \
                   lags = 1, \
                   trend='c', \
                   seasonal = False, \
                   exog = None, \
                   hold_back = None, \
                   period = None, \
                   missing = 'none')
fit_ag = model_ag.fit()
print("Coefficients:\n%s" % fit_ag.params)

In [ ]:
y_pred = fit_ag.predict(start=y_test.index[0], \
                        end=y_test.index[-1], \
                        dynamic=False)

Plot the prediction vs the training data

In [ ]:
import matplotlib.pyplot as plt
def plot_predictions(train, test, pred):
    plt.figure(figsize=(8,6))
    plt.plot(train.index, train.Total, label='Train')
    plt.plot(test.index, test, label='Test')
    plt.plot(pred.index, pred, label='Prediction')
    plt.yscale('log')
    plt.legend(loc='best')
    plt.title("Predictions vs input")
    plt.show()
plot_predictions(y_train, y_test, y_pred)

For comparison, get some scores for the model using a test set

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test.values, y_pred.values)

Out of interest, what do the predictions look like out into the future, say to 2040?

In [ ]:
from datetime import datetime
y_pred2 = fit_ag.predict(start=datetime(2016, 1, 1), \
                             end=datetime(2040, 1, 1), \
                             dynamic=False)

plot_predictions(y_train, y_test, y_pred2)

How do we then make a single prediction

In [ ]:
single_prediction = fit_ag.predict(start=datetime(2020, 1, 1), \
                             end=datetime(2020, 1, 1), \
                             dynamic=False)
single_prediction.values[0]

## Saving the model to disk

In [ ]:
import statsmodels.api as sm
model_path = "model.pickle"
fit_ag.save(model_path)
fit_ag2 = sm.load_pickle(model_path)
single_prediction2 = fit_ag2.predict(start=datetime(2020, 1, 1), \
                             end=datetime(2020, 1, 1), \
                             dynamic=False)
single_prediction2.values[0]